### ASP.NET COre HTTP Request Pipeline

The ASP.NET Core HTTP request pipeline is a series of middleware components that are invoked in a specific order. The pipeline is configured in the Startup class. The Startup class is where you configure the request pipeline and services that are used by your application. The request pipeline is configured by adding middleware components to an IApplicationBuilder instance that is provided by dependency injection.

-> Request -> Middleware -> Response


### Project Structure
E-Commerce API

- Token system to identify users
- Using Reddis for caching


#### Assignment
write a a middleware that would check if the request header has x-api-key

context.request.headers.containskey("x-api-key")
log header value